In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("cust_satisfaction.csv")
df.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied,13,460,5,4,5,25,18.0
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied,25,235,1,3,1,1,6.0
2,Female,Loyal Customer,Business travel,Business,satisfied,26,1142,5,4,5,0,0.0
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied,25,562,2,3,2,11,9.0
4,Male,Loyal Customer,Business travel,Business,satisfied,61,214,3,4,3,0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Gender                      103904 non-null  object 
 1   Customer Type               103904 non-null  object 
 2   Type of Travel              103904 non-null  object 
 3   Class                       103904 non-null  object 
 4   satisfaction                103904 non-null  object 
 5   Age                         103904 non-null  int64  
 6   Flight Distance             103904 non-null  int64  
 7   Inflight entertainment      103904 non-null  int64  
 8   Baggage handling            103904 non-null  int64  
 9   Cleanliness                 103904 non-null  int64  
 10  Departure Delay in Minutes  103904 non-null  int64  
 11  Arrival Delay in Minutes    103594 non-null  float64
dtypes: float64(1), int64(6), object(5)
memory usage: 9.5+ MB


In [4]:
df.isnull().sum()
df.dropna(inplace=True)

In [5]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [6]:
df["Customer Type"].value_counts()

Customer Type
Loyal Customer       84517
disloyal Customer    18905
Name: count, dtype: int64

In [7]:
loyal_customer = df[df["Customer Type"] == "Loyal Customer"]
disloyal_customer = df[df["Customer Type"] == "disloyal Customer"]

In [8]:
loyal_customer = loyal_customer.sample(20000)
loyal_customer.shape

(20000, 12)

In [9]:
balance_df = pd.concat([loyal_customer, disloyal_customer], axis=0)
balance_df.shape

(38905, 12)

In [10]:
cat_col = balance_df.select_dtypes(include=["object"])
cat_col.head()

,Gender,Customer Type,Type of Travel,Class,satisfaction
51618,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
49333,Male,Loyal Customer,Business travel,Business,neutral or dissatisfied
75834,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
59520,Male,Loyal Customer,Business travel,Business,neutral or dissatisfied
87163,Female,Loyal Customer,Business travel,Eco,neutral or dissatisfied


In [11]:
num_col = balance_df.select_dtypes(exclude=["object"])
num_col.head()

,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
51618,16,646,5,5,5,0,0.0
49333,30,1566,1,3,1,79,115.0
75834,9,1072,3,4,3,37,33.0
59520,21,2676,3,3,3,0,0.0
87163,50,1065,3,4,4,214,194.0


In [12]:
pd.get_dummies(cat_col, drop_first=True).astype(int).head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
51618,0,0,1,1,0,0
49333,1,0,0,0,0,0
75834,0,0,1,1,0,0
59520,1,0,0,0,0,0
87163,0,0,0,1,0,0


In [13]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
ohe = OneHotEncoder(drop="if_binary")
cat_col_encoded = ohe.fit_transform(cat_col).toarray()
cat_col_encoded

array([[0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]], shape=(38905, 7))

In [14]:
column_name = list(ohe.get_feature_names_out())
column_name


['Gender_Male',
 'Customer Type_disloyal Customer',
 'Type of Travel_Personal Travel',
 'Class_Business',
 'Class_Eco',
 'Class_Eco Plus',
 'satisfaction_satisfied']

In [15]:
one_hot = pd.DataFrame(cat_col_encoded, columns=column_name)
one_hot.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
one_hot = one_hot.reset_index(drop=True)
num_col = num_col.reset_index(drop=True)
final_df = pd.concat([one_hot, num_col], axis=1)
final_df.head()

,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,satisfaction_satisfied,Age,Flight Distance,Inflight entertainment,Baggage handling,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,16,646,5,5,5,0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,30,1566,1,3,1,79,115.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,9,1072,3,4,3,37,33.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,21,2676,3,3,3,0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,50,1065,3,4,4,214,194.0


In [17]:
##x and y ---> train test split -->algo train
x = final_df.drop("Customer Type_disloyal Customer", axis=1)
y = final_df["Customer Type_disloyal Customer"]

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [19]:
##deep learning -- tensorflow keras
# pip install tensorflow


In [20]:
##standard scaler 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
x_train.shape[1]

13

In [24]:
##define your ann model 
model = Sequential()
##input layer 
##68 -- no of neurons -- general convention -- 128 
model.add(Dense(68,activation="relu",input_dim = x_train_scaled.shape[1]))
##hidden layer 
model.add(Dense(32, activation="relu"))
model.add(Dense(24, activation="relu"))
model.add(Dense(12, activation="relu"))
##output layer 
model.add(Dense(1, activation="sigmoid"))
##model compile 
model.compile(optimizer="adam",
               loss="binary_crossentropy",
                 metrics=["accuracy"])
model.summary()

c:\Users\language lab\Desktop\python\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 68)             │           952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,265 (16.66 KB)

 Trainable params: 4,265 (16.66 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# (13+1)* 68

# (68+1) *32

In [27]:
history = model.fit(x_train_scaled,y_train,
                    epochs = 10 ,
                    validation_data=(x_test_scaled,y_test))

Epoch 1/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.8659 - loss: 0.3336 - val_accuracy: 0.9086 - val_loss: 0.2352
Epoch 2/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9085 - loss: 0.2354 - val_accuracy: 0.9112 - val_loss: 0.2298
Epoch 3/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9134 - loss: 0.2203 - val_accuracy: 0.9094 - val_loss: 0.2286
Epoch 4/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9184 - loss: 0.2091 - val_accuracy: 0.9172 - val_loss: 0.2166
Epoch 5/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9186 - loss: 0.2107 - val_accuracy: 0.9163 - val_loss: 0.2151
Epoch 6/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9225 - loss: 0.1998 - val_accuracy: 0.9199 - val_loss: 0.2100
Epoch 7/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.9233 - loss: 0.1982 - val_accuracy: 0.9202 - val_loss: 0.2120
Epoch 8/10
973/973 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9280 - loss: 0.1880 - val_accu

In [28]:
## prediction 
y_pred = model.predict(x_test_scaled)
prediction_label = (y_pred > 0.5).astype(int).ravel()
prediction_label

244/244 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


array([1, 1, 1, ..., 0, 1, 0], shape=(7781,))

In [29]:
model.save("model_classfication.h5")

In [30]:
from tensorflow.keras.models import load_model
model_load = load_model("model_classfication.h5")